In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("idevji1/sherlock-holmes-stories")

print("Path to dataset files:", path)

100%|██████████| 9.93M/9.93M [00:00<00:00, 61.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/idevji1/sherlock-holmes-stories/versions/1


In [26]:
!pip install rouge

In [27]:
import numpy as np
import pandas as pd
import os
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag
from nltk.util import ngrams
from collections import defaultdict, Counter
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge


In [ ]:
nltk.download('all')

In [13]:
# Fix the path - remove the leading space and add a trailing slash
story_path = "/root/.cache/kagglehub/datasets/idevji1/sherlock-holmes-stories/versions/1/"

def read_all_stories(story_path):
    txt = []
    for root, _, files in os.walk(story_path):
        for file in files:
            file_path = os.path.join(root, file)  # Use os.path.join for proper path handling
            with open(file_path) as f:
                for line in f:
                    line = line.strip()
                    if line=='----------': break
                    if line!='':txt.append(line)
    return txt

stories = read_all_stories(story_path)
print("number of lines = ", len(stories))

number of lines =  430042


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
def clean_txt(txt):
    cleaned_txt = []
    for line in txt:
        line = line.lower()
        line = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "", line)
        tokens = word_tokenize(line)
        words = [word for word in tokens if word.isalpha()]
        cleaned_txt+=words
    return cleaned_txt

cleaned_stories = clean_txt(stories)
print("number of words = ", len(cleaned_stories))

number of words =  4664494


In [16]:
def build_markov_chain(text_corpus, order=2):
    """
    Creates a Markov chain model from a list of words.

    Args:
        text_corpus: List of words in the corpus
        order: The length of the n-gram to consider as state

    Returns:
        A dictionary representing the Markov model with transition probabilities
    """
    # Initialize the model
    model = {}

    # We need enough words to create both a state and a next token
    if len(text_corpus) <= order:
        return model

    # Build the model from the corpus
    for i in range(len(text_corpus) - order):
        # Create current state from n consecutive words
        current = tuple(text_corpus[i:i+order])

        # Get the next word
        next_word = text_corpus[i+order]

        # Add to model (initialize if needed)
        if current not in model:
            model[current] = {}

        if next_word not in model[current]:
            model[current][next_word] = 0

        # Increment count
        model[current][next_word] += 1

    # Convert counts to probabilities
    for state in model:
        total_transitions = sum(model[state].values())
        for next_word in model[state]:
            model[state][next_word] /= total_transitions

    return model

def generate_text(markov_model, seed_words, length=100):
    """
    Generate text using the Markov model.

    Args:
        markov_model: The Markov chain model
        seed_words: List of words to start the generation
        length: Number of words to generate

    Returns:
        A string of generated text
    """
    # Ensure we have a valid seed
    if len(seed_words) < len(next(iter(markov_model))):
        return "Seed text too short for this model"

    # Initialize with the seed words
    current = tuple(seed_words[-len(next(iter(markov_model))):])
    result = list(current)

    # Generate text
    for _ in range(length):
        if current not in markov_model:
            # If we reach a state with no transitions, break
            break

        # Choose the next word based on transition probabilities
        transitions = markov_model[current]
        next_word = random.choices(
            list(transitions.keys()),
            weights=list(transitions.values()),
            k=1
        )[0]

        # Add to result
        result.append(next_word)

        # Update current state by sliding window
        current = tuple(result[-len(current):])

    return ' '.join(result)

In [18]:
# Build the model
model = build_markov_chain(cleaned_stories, order=2)

# Generate text (starting with a seed like the first two words)
generated_text = generate_text(model, cleaned_stories[:2], length=100)
print(generated_text)

the greek e will break her heart when she wouldnt do it but with the small morocco casket in which all of us glimmering in the jefferson hope felt his head sadly well well you have a word von bork clutched at my wrist and endeavoured to sound the dirge of our time so kindly read it yourself mycroft you can make nothing until i am unduly singing my own province this has been until lately lonelier than ever i confess that i should like to tell me where is the only servant i noticed it mr trelawney hope dropped his goose


In [24]:
# Check the number of states
print("number of states = ", len(model.keys()))

# check transitions from a specific state
the_game_state = ('the', 'servant')
if the_game_state in model:
    print("All possible transitions from 'the servant' state: \n")
    print(model[the_game_state])
else:
    print("'the game' state not found in model")

# Generate stories using the generation function
# For example, to start with "dear holmes"
for i in range(20):
    # Convert string to list of words for the seed
    seed_words = "dear holmes".split()
    # Generate text
    story = generate_text(model, seed_words, length=8)
    print(f"{i}. {story}")

# Generate a longer story
case_seed = "the case".split()
long_story = generate_text(model, case_seed, length=100)
print(long_story)

number of states =  208717
All possible transitions from 'the servant' state: 

{'i': 0.08, 'there': 0.08, 'jose': 0.08, 'who': 0.08, 'had': 0.08, 'whats': 0.08, 'no': 0.08, 'and': 0.16, 'deposed': 0.08, 'pass': 0.06, 'seemed': 0.06, 'to': 0.08}
0. dear holmes if it is marvellous i trust that they
1. dear holmes am i to listen to the male relatives
2. dear holmes oh very well then what clue could you
3. dear holmes i have a key a moment to me
4. dear holmes i think you would note he slapped the
5. dear holmes what is his cap where was you you
6. dear holmes and i have a clear run here of
7. dear holmes if matters came to the lodge to employ
8. dear holmes if it had not that i see also
9. dear holmes i believe that the continued resistance of the
10. dear holmes that there is a most skilful rider the
11. dear holmes it is he cried can you pick any
12. dear holmes i took the dead woman well if you
13. dear holmes what is your own showing new to him
14. dear holmes that you have afflicted

In [31]:
class AdvancedMarkovModel:
    def __init__(self, max_order=3):
        """
        Initialize an advanced Markov model with multiple n-gram orders.

        Args:
            max_order: Maximum n-gram order to consider
        """
        self.max_order = max_order
        self.models = {}  # Will store models of different orders
        self.beginnings = []  # Sentence beginnings
        self.endings = []  # Sentence endings
        self.pos_transitions = defaultdict(Counter)  # POS tag transitions
        self.word_by_pos = defaultdict(list)  # Words by POS tag

    def preprocess_text(self, text):
        """
        Preprocesses raw text into sentences and words.

        Args:
            text: Raw text (list of lines or single string)

        Returns:
            List of sentences, where each sentence is a list of words
        """
        if isinstance(text, list):
            text = ' '.join(text)

        # Split into sentences
        sentences = sent_tokenize(text)
        processed_sentences = []

        for sentence in sentences:
            # Clean and tokenize the sentence
            clean_sentence = sentence.lower()
            clean_sentence = re.sub(r"[^\w\s\.\,\!\?\'\"]", "", clean_sentence)
            words = word_tokenize(clean_sentence)

            # Keep only sentences with actual content
            if len(words) > 3:
                processed_sentences.append(words)

                # Store sentence beginnings and endings
                self.beginnings.append(tuple(words[:min(self.max_order, len(words))]))
                self.endings.append(tuple(words[-min(self.max_order, len(words)):]))

        return processed_sentences

    def build_pos_model(self, sentences):
        """
        Builds a part-of-speech transition model.

        Args:
            sentences: List of sentences (each a list of words)
        """
        for sentence in sentences:
            # Tag parts of speech
            pos_tags = pos_tag(sentence)

            # Add words to POS dictionary
            for word, tag in pos_tags:
                self.word_by_pos[tag].append(word)

            # Build POS transitions
            for i in range(len(pos_tags) - 1):
                current_tag = pos_tags[i][1]
                next_tag = pos_tags[i + 1][1]
                self.pos_transitions[current_tag][next_tag] += 1

    def build_models(self, sentences):
        """
        Builds variable-order Markov models.

        Args:
            sentences: List of sentences (each a list of words)
        """
        # Build models for each order from 1 to max_order
        for order in range(1, self.max_order + 1):
            model = defaultdict(Counter)

            for sentence in sentences:
                # Skip sentences shorter than order+1
                if len(sentence) <= order:
                    continue

                # Create n-grams
                for i in range(len(sentence) - order):
                    state = tuple(sentence[i:i+order])
                    next_word = sentence[i+order]
                    model[state][next_word] += 1

            # Convert counts to probabilities
            for state, transitions in model.items():
                total = sum(transitions.values())
                for next_word, count in transitions.items():
                    model[state][next_word] = count / total

            self.models[order] = dict(model)

    def train(self, text):
        """
        Train the model on the provided text.

        Args:
            text: Raw text to train on
        """
        sentences = self.preprocess_text(text)
        self.build_pos_model(sentences)
        self.build_models(sentences)

    def generate_next_word(self, state, current_order):
        """
        Generate the next word using backoff mechanism.

        Args:
            state: Current state (tuple of words)
            current_order: Current n-gram order

        Returns:
            Next word
        """
        # Try to find the state in the current order model
        if current_order > 0 and state[-current_order:] in self.models[current_order]:
            transitions = self.models[current_order][state[-current_order:]]
            next_word = random.choices(
                list(transitions.keys()),
                weights=list(transitions.values()),
                k=1
            )[0]
            return next_word

        # Backoff to a lower order model
        elif current_order > 1:
            return self.generate_next_word(state, current_order - 1)

        # If all else fails, return a random word
        else:
            return random.choice([next_word for next_words in self.models[1].values()
                                 for next_word in next_words.keys()])

    def generate_pos_aware_word(self, prev_tag):
        """
        Generate a word based on part-of-speech constraints.

        Args:
            prev_tag: Previous POS tag

        Returns:
            Word and its POS tag
        """
        # Get possible next tags
        if prev_tag in self.pos_transitions:
            next_tags = list(self.pos_transitions[prev_tag].keys())
            weights = list(self.pos_transitions[prev_tag].values())
            next_tag = random.choices(next_tags, weights=weights, k=1)[0]
        else:
            next_tag = random.choice(list(self.word_by_pos.keys()))

        # Get a word for that tag
        if next_tag in self.word_by_pos and self.word_by_pos[next_tag]:
            word = random.choice(self.word_by_pos[next_tag])
        else:
            # Fallback
            all_words = [w for words in self.word_by_pos.values() for w in words]
            word = random.choice(all_words) if all_words else "the"

        return word, next_tag

    def generate_text(self, length=100, seed=None, use_pos=True):
        """
        Generate text using the trained model.

        Args:
            length: Number of words to generate
            seed: Optional starting words (list)
            use_pos: Whether to use POS information

        Returns:
            Generated text
        """
        if not self.models:
            return "Model not trained yet."

        # Initialize with seed or random beginning
        if seed and len(seed) > 0:
            current = tuple(seed[-self.max_order:] if len(seed) >= self.max_order else seed)
            result = list(seed)
        else:
            current = random.choice(self.beginnings)
            result = list(current)

        current_pos = pos_tag(result)[-1][1] if use_pos and result else "NN"

        # Generate text
        i = 0
        while i < length:
            # Check if we should end the sentence
            if tuple(result[-min(len(result), self.max_order):]) in self.endings and i > length // 2:
                if len(result) > 0 and result[-1][-1] not in ".!?":
                    result[-1] = result[-1] + "."
                break

            # Get next word
            if use_pos and random.random() < 0.7:  # 70% chance to use POS model
                next_word, current_pos = self.generate_pos_aware_word(current_pos)
            else:
                next_word = self.generate_next_word(tuple(result[-self.max_order:]), self.max_order)

            # Add word to result
            result.append(next_word)
            i += 1

            # Update current state
            current = tuple(result[-self.max_order:])

        # Ensure sentence ends with punctuation
        if len(result) > 0 and result[-1][-1] not in ".!?":
            result[-1] = result[-1] + "."

        # Apply some basic post-processing to fix issues
        text = ' '.join(result)
        text = re.sub(r'\s+([.,!?])', r'\1', text)  # Fix spacing before punctuation
        text = re.sub(r'\.+', '.', text)  # Remove multiple periods
        text = re.sub(r'\s+', ' ', text)  # Remove extra spaces

        # Capitalize first letter of sentences
        sentences = re.split(r'([.!?])', text)
        processed_text = ""
        for i in range(0, len(sentences) - 1, 2):
            if sentences[i].strip():
                sentence = sentences[i].strip()
                sentence = sentence[0].upper() + sentence[1:]
                processed_text += sentence + sentences[i+1] + " "

        return processed_text.strip()

    def evaluate_bleu(self, reference_texts, generated_text, n=4):
        """
        Evaluate generated text using BLEU score.

        Args:
            reference_texts: List of reference texts
            generated_text: Generated text to evaluate
            n: Maximum n-gram order to consider

        Returns:
            BLEU score
        """
        # Tokenize texts
        references = [word_tokenize(text.lower()) for text in reference_texts]
        candidate = word_tokenize(generated_text.lower())

        # Calculate BLEU score with smoothing
        weights = [1/n] * n
        smoothing = SmoothingFunction().method1
        return sentence_bleu(references, candidate, weights=weights, smoothing_function=smoothing)

    def evaluate_rouge(self, reference_text, generated_text):
        """
        Evaluate generated text using ROUGE score.

        Args:
            reference_text: Reference text
            generated_text: Generated text to evaluate

        Returns:
            ROUGE scores (dictionary)
        """
        rouge = Rouge()
        scores = rouge.get_scores(generated_text, reference_text)
        return scores[0]

    def calculate_perplexity(self, test_text, order=2):
        """
        Calculate perplexity of the model on test text.

        Args:
            test_text: Text to evaluate on
            order: N-gram order to use

        Returns:
            Perplexity score (lower is better)
        """
        if order not in self.models:
            return float('inf')

        sentences = self.preprocess_text(test_text)
        log_prob_sum = 0
        word_count = 0

        for sentence in sentences:
            if len(sentence) <= order:
                continue

            word_count += len(sentence) - order

            for i in range(len(sentence) - order):
                state = tuple(sentence[i:i+order])
                next_word = sentence[i+order]

                # Calculate probability with smoothing
                if state in self.models[order] and next_word in self.models[order][state]:
                    prob = self.models[order][state][next_word]
                else:
                    # Smoothing: use a small probability if transition not found
                    prob = 1e-10

                log_prob_sum += np.log2(prob)

        if word_count == 0:
            return float('inf')

        perplexity = 2 ** (-log_prob_sum / word_count)
        return perplexity



In [32]:
def demonstrate_advanced_markov(text_data):
    # Initialize and train the model
    model = AdvancedMarkovModel(max_order=3)
    model.train(text_data)

    # Generate text with different seeds
    print("Generated text with random seed:")
    generated1 = model.generate_text(length=150)
    print(generated1)
    print("\n")

    print("Generated text with seed 'Sherlock Holmes':")
    generated2 = model.generate_text(length=150, seed=["Sherlock", "Holmes"])
    print(generated2)
    print("\n")

    # Evaluate the model
    print("Evaluation metrics:")
    print("--------------------")
    # Split the text into train/test
    sentences = model.preprocess_text(text_data)
    random.shuffle(sentences)
    split = int(0.8 * len(sentences))
    train_sentences = sentences[:split]
    test_sentences = sentences[split:]

    # Convert back to text for evaluation
    test_text = " ".join([" ".join(sentence) for sentence in test_sentences])

    # Calculate perplexity
    perplexity = model.calculate_perplexity(test_text)
    print(f"Perplexity: {perplexity:.2f}")

    # Calculate BLEU
    reference_texts = [" ".join(sentence) for sentence in test_sentences[:5]]
    bleu_score = model.evaluate_bleu(reference_texts, generated1)
    print(f"BLEU score: {bleu_score:.4f}")

    # Calculate ROUGE if test_sentences are not empty
    if test_sentences:
        reference_text = " ".join([" ".join(sentence) for sentence in test_sentences[:10]])
        rouge_scores = model.evaluate_rouge(reference_text, generated1)
        print(f"ROUGE-1 F1: {rouge_scores['rouge-1']['f']:.4f}")
        print(f"ROUGE-2 F1: {rouge_scores['rouge-2']['f']:.4f}")
        print(f"ROUGE-L F1: {rouge_scores['rouge-l']['f']:.4f}")

    return model



In [33]:
advanced_model = demonstrate_advanced_markov(stories)

Generated text with random seed:
`` i 'm sorry, holmes. '' left so nearly cost us, and marked and the as the 'can day and an exceedingly sir that fright. Had to tell the direction dreadful, finding of the all the left stick, and macdonald 's night, and many being the same caltrops staggered it was several sort and said? '' '' was ever had. As other would have all sheridan operations, friend for the preference that it, and, with ten its i was never, dying man.


Generated text with seed 'Sherlock Holmes':
Sherlock Holmes, whose 'tweendecks of possible family, and once often to try to have. '' succeeded he approached this lamp watson, even and for a long time that shoulders, bent at after them. Must look tell to miss see that a fellow that i never he was have it single it cut light upon and very seldom it seldom seemed touch. ' '' corresponded brown yourself went a bit so. '.


Evaluation metrics:
--------------------
Perplexity: 13.94
BLEU score: 0.0099
ROUGE-1 F1: 0.2154
ROUGE-2 F1: 0.